# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [1]:
from __future__ import division
import graphlab
import math
import string
import pandas as pd

A newer version of GraphLab Create (v1.8.5) is available! Your current version is v1.8.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [2]:
products = graphlab.SFrame('amazon_baby.gl/amazon_baby.gl/')

[INFO] This non-commercial license of GraphLab Create is assigned to thmho@uwaterloo.ca and will expire on February 16, 2017. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1900 - Server binary: C:\Program Files (x86)\Anaconda\lib\site-packages\graphlab\unity_server.exe - Server log: C:\Users\Michael\AppData\Local\Temp\graphlab_server_1459115708.log.0
[INFO] GraphLab Server Version: 1.8.1


In [6]:
pproducts = products.to_dataframe()

Now, let us see a preview of what the dataset looks like.

In [4]:
pproducts

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [5]:
print pproducts.loc[269,:]

name      The First Years Massaging Action Teether
review                    A favorite in our house!
rating                                           5
Name: 269, dtype: object


Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [3]:
def remove_punctuation(text):
    return text.translate(None, string.punctuation) 

In [7]:
pproducts = pproducts.fillna({'review':''})
pproducts['review_clean'] = pproducts['review'].apply(remove_punctuation)



Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [8]:
from collections import Counter
ex = pproducts.iloc[205]['review_clean']
print ex
print ex.lower().split()
cnt = Counter()
for word in ex.split():
    cnt[word]+=1
cnt

My 8 month toddler enjoys this book Very good idea with mindstimulating activities on each page Well worth it for the price
['my', '8', 'month', 'toddler', 'enjoys', 'this', 'book', 'very', 'good', 'idea', 'with', 'mindstimulating', 'activities', 'on', 'each', 'page', 'well', 'worth', 'it', 'for', 'the', 'price']


Counter({'8': 1,
         'My': 1,
         'Very': 1,
         'Well': 1,
         'activities': 1,
         'book': 1,
         'each': 1,
         'enjoys': 1,
         'for': 1,
         'good': 1,
         'idea': 1,
         'it': 1,
         'mindstimulating': 1,
         'month': 1,
         'on': 1,
         'page': 1,
         'price': 1,
         'the': 1,
         'this': 1,
         'toddler': 1,
         'with': 1,
         'worth': 1})

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
pproducts = pproducts[pproducts['rating'] != 3]
len(pproducts)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [10]:
pproducts['sentiment'] = pproducts['rating'].apply(lambda rating : +1 if rating > 3 else -1)
pproducts

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1
6,A Tale of Baby's Days with Peter Rabbit,"Lovely book, it's bound tightly so you may not...",4,Lovely book its bound tightly so you may not b...,1
7,"Baby Tracker&reg; - Daily Childcare Journal, S...",Perfect for new parents. We were able to keep ...,5,Perfect for new parents We were able to keep t...,1
8,"Baby Tracker&reg; - Daily Childcare Journal, S...",A friend of mine pinned this product on Pinter...,5,A friend of mine pinned this product on Pinter...,1
9,"Baby Tracker&reg; - Daily Childcare Journal, S...",This has been an easy way for my nanny to reco...,4,This has been an easy way for my nanny to reco...,1
10,"Baby Tracker&reg; - Daily Childcare Journal, S...",I love this journal and our nanny uses it ever...,4,I love this journal and our nanny uses it ever...,1


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [11]:
train_data, test_data = products.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

146786
36745


In [12]:
ptrain_data = train_data.to_dataframe()
ptrain_data = ptrain_data.fillna({'review':''})
ptrain_data['review_clean'] = ptrain_data['review'].apply(remove_punctuation)
ptrain_data['word_count'] = ptrain_data['review_clean'].apply(lambda x: len(x.split()))
ptrain_data = ptrain_data[ptrain_data['rating'] != 3]
ptrain_data['sentiment'] = ptrain_data['rating'].apply(lambda rating : +1 if rating > 3 else -1)

ptest_data = test_data.to_dataframe()
ptest_data = ptest_data.fillna({'review':''})
ptest_data['review_clean'] = ptest_data['review'].apply(remove_punctuation)
ptest_data['word_count'] = ptest_data['review_clean'].apply(lambda x: len(x.split()))
ptest_data = ptest_data[ptest_data['rating'] != 3]
ptest_data['sentiment'] = ptest_data['rating'].apply(lambda rating : +1 if rating > 3 else -1)

In [13]:
type(ptrain_data['review_clean'])

pandas.core.series.Series

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
# Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(ptrain_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(ptest_data['review_clean'])

In [14]:
print ptrain_data
print ptest_data

                                                     name  \
1                                   Planetwise Wipe Pouch   
2                     Annas Dream Full Quilt with 2 Shams   
3       Stop Pacifier Sucking without tears with Thumb...   
4       Stop Pacifier Sucking without tears with Thumb...   
5       Stop Pacifier Sucking without tears with Thumb...   
6                 A Tale of Baby's Days with Peter Rabbit   
7       Baby Tracker&reg; - Daily Childcare Journal, S...   
8       Baby Tracker&reg; - Daily Childcare Journal, S...   
9       Baby Tracker&reg; - Daily Childcare Journal, S...   
10      Baby Tracker&reg; - Daily Childcare Journal, S...   
12         Nature's Lullabies First Year Sticker Calendar   
13         Nature's Lullabies First Year Sticker Calendar   
14        Nature's Lullabies Second Year Sticker Calendar   
15        Nature's Lullabies Second Year Sticker Calendar   
16        Nature's Lullabies Second Year Sticker Calendar   
17        Nature's Lulla

In [15]:
bag_of_words = []
for i in ptrain_data['review_clean']:
    bag_of_words.append(i)
print bag_of_words[0:2]

['it came early and was not disappointed i love planet wise bags and now my wipe holder it keps my osocozy wipes moist and does not leak highly recommend it', 'Very soft and comfortable and warmer than it looksfit the full size bed perfectlywould recommend to anyone looking for this type of quilt']


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [16]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)

ToolkitError: The following columns were expected but are missing: ['sentiment']

In [17]:
len(vectorizer.get_feature_names())

121338

In [15]:
from sklearn import linear_model

sentiment_model = linear_model.LogisticRegression()
sentiment_model.fit(train_matrix, ptrain_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [17]:
weights = sentiment_model.coef_

weights.size

121338

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [18]:
num_positive_weights = (weights[weights > 0]).size
num_negative_weights = (weights[weights < 0]).size

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 86920 
Number of negative weights: 34418 


**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [19]:
sample_test_data = ptest_data[10:13]
print sample_test_data['rating']
sample_test_data

11    5
12    5
13    5
Name: rating, dtype: float64


,name,review,rating,review_clean,word_count,sentiment
11,Our Baby Girl Memory Book,Really happy with this purchase. I was looking...,5,Really happy with this purchase I was looking ...,42,1
12,Cloth Diaper Pins Stainless Steel Traditional ...,It has been many years since we needed diaper ...,5,It has been many years since we needed diaper ...,87,1
13,Newborn Baby Tracker&reg; - Round the Clock Ch...,Love it love it love it!! Got my first baby t...,5,Love it love it love it Got my first baby tra...,71,1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [20]:
sample_test_data.iloc[0]['review']

"Really happy with this purchase. I was looking for an english baby book (I'm French and the daddy is Flammish) and this is the perfect one. Lot of info to fill in : family, firsts...and mini pockets to preserve hair, maternity bracelet...Really happy!"

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [21]:
sample_test_data.iloc[1]['review']

"It has been many years since we needed diaper pins, but I do like to pin my socks together through the wash and have never lost one yet. I was using brass dressmaker pins but they tended to open up during the wash cycle. My wife suggested diaper pins and we tried them. Thery're working perfectly and staying together through the washing. Matter of fact they can get a little tricky to open, but of course that was the idea. Recommended to keep your socks &#34;friended up&#34;."

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [22]:
sample_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_matrix)
print scores

[  9.82663394   7.87386249  10.92469517]


In [62]:
predictions_proba = sentiment_model.predict_proba(sample_matrix)
print predictions_proba

[[  5.39912840e-05   9.99946009e-01]
 [  3.80416833e-04   9.99619583e-01]
 [  1.80076628e-05   9.99981992e-01]]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [26]:
def predict_class(scores):
    return [1 if score > 0 else 0 for score in scores]
    

Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [27]:
predict_class(scores)

[1, 1, 1]

In [99]:
p = sentiment_model.predict(sample_matrix)
(p == [1,1,0]).sum()


2

**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [23]:
def probability_predictions(scores):
    return [1/(1 + math.exp(-score)) for score in scores]

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [24]:
probability_predictions(scores)

[0.9999460087159705, 0.9996195831665947, 0.9999819923372046]

In [ ]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(sample_test_data, output_type='probability')

In [ ]:
The second one

** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [63]:
ptest_matrix = vectorizer.transform(ptest_data['review_clean'])
prediction_prob = sentiment_model.predict_proba(ptest_matrix)


In [80]:
prediction_prob.shape

(133373L, 2L)

In [72]:
ptest_data["probability of 1"] = prediction_prob[:,1]
df = ptest_data.sort_values("prob")
df.loc[df['name'] == 'Safety 1st High-Def Digital Monitor']

,name,review,rating,review_clean,word_count,sentiment,prob,probability of 1
10631,Safety 1st High-Def Digital Monitor,We're on our second child with this monitor an...,5,Were on our second child with this monitor and...,165,1,0.983334,0.983334
10628,Safety 1st High-Def Digital Monitor,"Finally, a baby monitor without static. This ...",5,Finally a baby monitor without static This pr...,61,1,0.989316,0.989316
10633,Safety 1st High-Def Digital Monitor,I did loads of research on buying this monitor...,5,I did loads of research on buying this monitor...,85,1,0.995846,0.995846
10625,Safety 1st High-Def Digital Monitor,If you have a house-load of electronic gadgetr...,5,If you have a houseload of electronic gadgetry...,318,1,0.999432,0.999432
10630,Safety 1st High-Def Digital Monitor,Not everything worked as described in the ad b...,5,Not everything worked as described in the ad b...,52,1,0.999894,0.999894
10626,Safety 1st High-Def Digital Monitor,"First of all, ONE THOUSAND THANKS to the perso...",4,First of all ONE THOUSAND THANKS to the person...,336,1,0.999913,0.999913
10632,Safety 1st High-Def Digital Monitor,I have 7 kids so I been through A LOT of monit...,5,I have 7 kids so I been through A LOT of monit...,54,1,0.999999,0.999999
10627,Safety 1st High-Def Digital Monitor,"I kept debating buying the Philips, but had a ...",5,I kept debating buying the Philips but had a h...,225,1,0.999999,0.999999
10629,Safety 1st High-Def Digital Monitor,We've had this monitor for almost exactly a ye...,4,Weve had this monitor for almost exactly a yea...,244,1,1.000000,1.000000


In [27]:
df.iloc[-20:-1,:]

,name,review,rating,review_clean,word_count,sentiment,prob
11555,Graco TurboBooster Car Seat Hearts Delight,"My 3 year old was ready for a ""big girl car s...",5,My 3 year old was ready for a big girl car se...,398,1,1
22254,"Baby Jogger 2011 City Mini Double Stroller, Bl...",The wait is over; we have the new 2011 City Mi...,5,The wait is over we have the new 2011 City Min...,544,1,1
15942,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,424,1,1
32192,Safety 1st Alpha Omega Elite Convertible 3-in-...,My dad just bought this car seat for me/my son...,5,My dad just bought this car seat for memy son ...,1028,1,1
30381,"UPPAbaby Vista Stroller, Denny",I researched strollers for months and months b...,5,I researched strollers for months and months b...,440,1,1
19658,Jeep Overland Limited Jogging Stroller with Fr...,OK first off - I want to say that I did a LOT ...,5,OK first off I want to say that I did a LOT o...,365,1,1
33364,Britax Pavilion 70-G3 Convertible Car Seat Sea...,We LOVE this seat! As parents to 8 children ra...,5,We LOVE this seat As parents to 8 children ran...,522,1,1
32439,HALO SleepSack SwaddleChange Diaper Pad Covers...,"I love having a new baby in the house, and eve...",5,I love having a new baby in the house and even...,492,1,1
33391,"Inglesina 2013 Trip Stroller, Lampone Purple",I did many hours of research reading reviews a...,5,I did many hours of research reading reviews a...,721,1,1
10113,"Joovy Zoom 360 Swivel Wheel Jogging Stroller, ...",The joovy zoom 360 was the perfect solution fo...,5,The joovy zoom 360 was the perfect solution fo...,867,1,1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

In [93]:
df.iloc[1:20,:]

,name,review,rating,review_clean,word_count,sentiment,prob
8149,Arms Reach Co-Sleeper brand Mini Co-Sleeper Ba...,"Please see my email to the company:Hello,I am ...",1,Please see my email to the companyHelloI am wr...,664,-1,2.713049e-19
140125,"Zooper Twist Escape Stroller, Summer Day",I had to return this stroller for three reason...,1,I had to return this stroller for three reason...,739,-1,7.472102e-18
12830,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,332,-1,1.964194e-16
142681,iPad Travel Case,I have rated and left reviews for many items o...,1,I have rated and left reviews for many items o...,669,-1,2.815916e-16
106687,Summer Infant Sleek and Secure Hand Held Video...,The first monitor broke within 1 month of use-...,1,The first monitor broke within 1 month of use ...,326,-1,3.985644e-16
72039,"Peg-Perego Aria Twin Stroller, Java",I am so incredibly disappointed with the strol...,1,I am so incredibly disappointed with the strol...,487,-1,1.052162e-15
96193,Levana Safe N'See Digital Video Baby Monitor w...,I have NEVER written a review before for anyth...,1,I have NEVER written a review before for anyth...,949,-1,2.145774e-15
45827,"Dream On Me Bassinet, Blue",My husband and I are VERY disappointed and sho...,1,My husband and I are VERY disappointed and sho...,600,-1,2.839424e-15
74152,"Munchkin Arm and Hammer Diaper Pail, White",I would recommend in the strongest possible wa...,1,I would recommend in the strongest possible wa...,944,-1,3.030360e-15
32209,Ameda Purely Yours Breast Pump - Carry All,"I received this pump through insurance, and I ...",1,I received this pump through insurance and I f...,537,-1,1.499337e-14


## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [60]:
def get_classification_accuracy(model, data, true_labels, vectorizer):
    # First get the predictions
    data_matrix = vectorizer.transform(data['review_clean'])
    predictions = model.predict(data_matrix)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    correct = (predictions == true_labels).sum()

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = correct / len(true_labels)
    
    return accuracy

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [61]:
get_classification_accuracy(sentiment_model, ptest_data, ptest_data['sentiment'], vectorizer)

0.93328140447586805

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [102]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [103]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [104]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(ptrain_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(ptest_data['review_clean'])

In [ ]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [ ]:
train_data[0]['review']

The **word_count** column had been working with before looks like the following:

In [ ]:
print train_data[0]['word_count']

Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [ ]:
print train_data[0]['word_count_subset']

## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [ ]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

In [106]:
simple_model = linear_model.LogisticRegression()
simple_model.fit(train_matrix_word_subset, ptrain_data['sentiment'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [111]:
get_classification_accuracy(simple_model, ptest_data, ptest_data['sentiment'], vectorizer_word_subset)

0.86644973120496649

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [112]:
simple_model.coef_

array([[ 1.35896589,  0.93923446,  1.19655912,  0.11165472,  0.49885179,
         1.50539082,  1.76861089,  0.49588147,  0.19023108,  0.08167841,
        -1.70365555, -0.17342471, -0.5285127 , -2.00713727, -2.36498172,
        -0.64421402, -0.3064419 , -0.90550464, -0.33775057, -2.04464581]])

In [114]:
len(simple_model.coef_[0])

20

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [ ]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

In [121]:
from operator import itemgetter
simple_coef = sorted(zip(significant_words,simple_model.coef_[0]), key=itemgetter(1))

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [153]:
positive_simple_coef = [coef[0] for coef in simple_coef if coef[1] > 0]
print positive_simple_coef

['car', 'old', 'able', 'well', 'little', 'great', 'easy', 'love', 'perfect', 'loves']


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [135]:
print len(sentiment_model.coef_[0])
print len(vectorizer.get_feature_names())

121338
121338


In [163]:
full_coef = sorted(zip(vectorizer.get_feature_names(), sentiment_model.coef_[0]), key=itemgetter(1))

[coef for coef in full_coef if coef[0] in positive_simple_coef]


[(u'old', 0.062436363584517936),
 (u'car', 0.24415155935549493),
 (u'able', 0.31442240463067217),
 (u'well', 0.51208646317478412),
 (u'little', 0.59400847730078488),
 (u'great', 1.2475222485947519),
 (u'easy', 1.3899925635865233),
 (u'love', 1.5827552961036522),
 (u'loves', 1.6960665642240296),
 (u'perfect', 1.8874970355481746)]

In [164]:
print full_coef[-20:-1]

[(u'awesome', 1.7390630398885076), (u'highly', 1.7402229302256043), (u'perfectly', 1.7737692209709846), (u'sooner', 1.7754312750830783), (u'minor', 1.8112541864880409), (u'skeptical', 1.8113553276622709), (u'girly', 1.8331591576905182), (u'exactly', 1.8520758492363809), (u'outstanding', 1.8534650924155314), (u'pleasantly', 1.8751627161784072), (u'perfect', 1.8874970355481746), (u'saves', 1.8978278064174068), (u'complain', 1.9038613517663852), (u'worry', 2.0020770183154046), (u'penny', 2.0253084180079539), (u'con', 2.0349460725553565), (u'thankful', 2.0673163846498879), (u'lifesaver', 2.1107958844352721), (u'rich', 2.1111695514879614)]


# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [165]:
get_classification_accuracy(sentiment_model, ptrain_data, ptrain_data['sentiment'], vectorizer)

0.96903421232183429

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [166]:
get_classification_accuracy(simple_model, ptrain_data, ptrain_data['sentiment'], vectorizer_word_subset)

0.86644973120496649

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [170]:
get_classification_accuracy(sentiment_model, ptest_data, ptest_data['sentiment'], vectorizer)

0.93328140447586805

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [171]:
get_classification_accuracy(simple_model, ptest_data, ptest_data['sentiment'], vectorizer_word_subset)

0.87066718595524129

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [173]:
num_positive  = (ptrain_data['sentiment'] == +1).sum()
num_negative = (ptrain_data['sentiment'] == -1).sum()
print num_positive
print num_negative

112078
21295


Positive sentiment is the majority class in the train_data

Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [174]:
print(ptest_data['sentiment'] == +1).sum() / len(ptest_data['sentiment'])

0.844273345517


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?